# Path profiling with sketches

If we want to characterize how process control moves through a graph over time, _path profiling_ is a general approach that captures detail that node or edge profiling might miss.  Depending on the concrete application of path profiling, several engineering constraints may be more or less important:  do we want to count a small number of paths quickly and precisely or a much larger number of paths scalably?

The classic approach of [Ball and Larus](https://dl.acm.org/citation.cfm?id=243857) ([pdf](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.138.7451&rep=rep1&type=pdf)) provides an efficient technique for tracking acyclic intraprocedural paths through the control-flow graphs (CFG) of compiled programs.  Essentially, each possible path is represented by an integer, which is constructed by setting bits on block entry.  (Their technique constructs a spanning tree of the CFG so that it doesn't have to set bits after _every_ branch.)  The path numbering then indexes into an array of counters (for functions with a small number of possible paths) or into a (precise) hash table of counters (for functions with a large number of possible paths but sparsely-distributed actual paths).

In this notebook, we'll evaluate path profiling for a different problem with different constraints:  profiling the top paths to exceptional states in synthetic business process flowcharts.  The charts resemble function control-flow graphs but may have substantially more nodes than a typical compiled function and (especially since we may want to explicitly consider cycles, at least up to a certain loop count) substantially longer paths.  We also would like to maintain a separate profile for the paths leading to each terminal node, so that we can identify the most likely ways to get to a particular exceptional state and use this information to refine our processes.  Since compiled functions are typically small and the basic blocks within these may only have fewer than ten instructions, the cost of explicitly updating a counter upon entering a block (let alone building a dynamic structure to represent a path) may be prohibitive.  By contrast, the processes we're interested in modeling are made up of nodes that take longer to execute than a basic block, so we can maintain a dynamic path until a terminal node -- the problem is that we may not be able to precisely count the number of paths we've encountered.

We'll handle this problem by using a sketch to track the top *k* paths to each terminal node.

In [ ]:
import networkx as nx
from ipysigma import Sigma

In [ ]:
def synthetic_flowchart(size=100):
    import numpy as np
    
    def insert_step(g, f, t, n):
        """ inserts n as an extra node between f and t; removes an existing edge between f and t """
        g.add_edge(f, n)
        g.add_edge(n, t)
        g.remove_edge(f, t)
        
    def insert_branch(g, f, t, n):
        """ inserts n as an extra node between f and t """
        g.add_edge(f, n)
        g.add_edge(n, t)
    
    def insert_leaf(g, f, n):
        """ inserts n as a leaf node with an edge from f """
        g.add_edge(f, n)
        
    def create_loop(g, f, t):
        """ inserts a back edge from t to f """
        g.add_edge(t, f)
    
    def choose_edge(g):
        # mea maxima culpa
        edges = list(g.edges())
        return edges[np.random.randint(len(edges))]
    
    def choose_node(g):
        nodes = list(g)
        return np.random.choice(nodes)
        
    g = nx.DiGraph()
    g.add_edge(0, 1)
    
    next_node = 2
    while next_node <= size:
        edges = g.edges()
        nodes = g.nodes()
        which = np.random.randint(100)
        if which <= 30:
            f, t = choose_edge(g)
            insert_step(g, f, t, next_node)
            next_node = next_node + 1
        elif which <= 60:
            f, t = choose_edge(g)
            insert_branch(g, f, t, next_node)
            next_node = next_node + 1
        elif which <= 80:
            f, t = choose_edge(g)
            create_loop(g, f, t)
        else:
            insert_leaf(g, choose_node(g), next_node)
            next_node = next_node + 1
    
    return g

In [ ]:
g = synthetic_flowchart()

In [ ]:
Sigma(g)

We can generate random paths through our random graph.

In [ ]:
def path(g, bound=100):
    """ find a random path of at most size bound through a directed graph """
    import numpy as np
    
    path = [0]
    plen = 1
    while plen < bound:
        edges = list(g.edges(path[-1]))
        if len(edges) == 0:
            return path
        c, n = edges[np.random.randint(len(edges))]
        path.append(n)
        plen = plen + 1
    return path

def unipath(g, bound=100):
    """ find a random path of at most size bound through a directed graph; 
        return a representation of the path as a Unicode string (so it's hashable) """
    p = path(g, bound)
    return "".join([chr(0x1f600 + c) for c in p])

def deunipath(p):
    """ translate a unicode representation of a path into a list representation of that path """
    return [ord(c) - 0x1f600 for c in p]

We can also turn each path into a string  (Storing paths as strings will be necessary so that we can hash them and count them in a sketch.)

In [ ]:
unipath(g)

# Sketching paths

We'll use a count-min sketch paired with a priority queue in order to track the top ten paths over many simulations.  The actual count-min sketch implementation isn't important, but we'll include it here in case you want to dig in.

In [ ]:
from hashlib import sha1
import pickle
import numpy

def hashes_for(count, stride):
    def hashes(value):
        bvalue = type(value) == bytes and value or pickle.dumps(value)
        digest = sha1(bvalue).hexdigest()
        return [int(digest[s:s+stride], 16) for s in [x * stride for x in range(count)]]
    return hashes

class CMS(object):
    def __init__(self, width, hashes):
        """ Initializes a Count-min sketch with the
            given width and a collection of hashes, 
            which are functions taking arbitrary 
            values and returning integers.  The depth
            of the sketch structure is taken from the
            number of supplied hash functions.
            
            hashes can be either a function taking 
            a value and returning a list of results
            or a list of functions.  In the latter 
            case, this constructor will synthesize 
            the former """
        self.__width = width
        
        if hasattr(hashes, '__call__'):
            self.__hashes = hashes
            # inspect the tuple returned by the hash function to get a depth
            self.__depth = len(hashes(bytes()))
        else:
            funs = hashes[:]
            self.__depth = len(hashes)
            def h(value):
                return [int(f(value)) for f in funs]
            self.__hashes = h
        
        self.__buckets = numpy.zeros((int(width), int(self.__depth)), numpy.uint64)
    
    
    def width(self):
        return self.__width
    
    def depth(self):
        return self.__depth
    
    def insert(self, value):
        """ Inserts a value into this sketch """
        for (row, col) in enumerate(self.__hashes(value)):
            self.__buckets[col % self.__width][row] += 1
    
    def lookup(self, value):
        """ Returns a biased estimate of number of times value has been inserted in this sketch"""
        return min([self.__buckets[col % self.__width][row] for (row, col) in enumerate(self.__hashes(value))])
    
    def merge_from(self, other):
        """ Merges other in to this sketch by 
            adding the counts from each bucket in other
            to the corresponding buckets in this
            
            Updates this. """
        self.__buckets += other.__buckets
    
    def merge(self, other):
        """ Creates a new sketch by merging this sketch's
            counts with those of another sketch. """
        
        cms = CMS(self.width(), self.__hashes)
        cms.__buckets += self.__buckets
        cms.__buckets += other.__buckets
        return cms

    def dup(self):
        cms = CMS(self.width(), self.__hashes)
        cms.merge_from(self)
        return cms

Now we'll set up a simulation:

In [ ]:
import heapq
def simulate(g, trials=100000, topk=10):
    pq = []
    topset = set()
    cms = CMS(16384, hashes_for(3,8))
    for i in range(trials):
        path = unipath(g)
        cms.insert(path.encode())
        count = cms.lookup(path.encode())
        update = (count, path)
        if path in topset:
            pq = [(p == path and count or c, p) for (c, p) in pq]
            heapq.heapify(pq)
        else:
            topset.add(path)
            if len(topset) > topk:
                (c, remove) = heapq.heappushpop(pq, update)
                topset.remove(remove)
            else:
                heapq.heappush(pq, update)
    return pq
                    

In [ ]:
simulate(g, 100000)